# Задача: классификация твитов по тональности

1. [Скачать датасет с положительными и отрицательными твитами, разбить данные на тренировочную и тестовую выборки.](#загрузка-датасетов)
2. [Провести векторизацию слов в тексте на основе ngrams, построить pipeline для обучения простой модели классификации.](#векторизация-слов-на-основе-ngrams-и-обучение-модели-классификации)
3. [Повторить задачу классификации, вместо униграмм использовать триграммы.](#векторизация-слов-на-основе-триграмм)
4. [Повторить задачу классификации, но с векторизацией на основе TF-IDF.](#векторизация-слов-на-основе-tf-idf)
5. [Выполнить токенизацию текста с помощью нескольких разных токенизаторов, сравнить их работу.](#токенизация-текста-с-помощью-нескольких-разных-токенизаторов)
6. [Выполнить лемматизацию (сведение слов к начальной -- словарной форме).](#лемматизация)
7. [Повторить задачу классификации, использовать униграммы на основе отдельных символов.](#эксплоративный-анализ)
8. [Исследовать роль регулярных выражений в предобработке.](#регулярные-выражения)

Результаты всех моделей показаны здесь: [Результаты моделей](#результаты-моделей)

In [2]:
from codecs import replace_errors

FIRST_TIME = False

## Загрузка датасетов

In [3]:
if FIRST_TIME:
    !wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
    !wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

In [4]:
import pandas as pd
import numpy as np

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [6]:
if FIRST_TIME:
    import os
    os.chdir('./LanguageProcessing')

# загружаем положительные твиты
positive = pd.read_csv('./positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive) # устанавливаем метки

# загружаем отрицательные твиты
negative = pd.read_csv('./negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative) # устанавливаем метки

# соединяем вместе
df = pd.concat([positive , negative])
df.describe()

,text,label
count,226834,226834
unique,217440,2
top,Офигенный день!\nдень позитива)\nбегал как иди...,positive
freq,156,114911


### Разбиение на обучающую и тестовую выборки

In [7]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((170125,), (170125,), (56709,), (56709,))

### Функция сохраняющая результаты моделей с названиями моделей

In [8]:
models_df = pd.DataFrame(columns=['name', 'accuracy'])

def save_model_result(name, accuracy):
    global models_df
    models_df.loc[len(models_df)] = [name, accuracy]

def show_ranked_models():
    global models_df
    models_df = models_df.sort_values(by='accuracy', ascending=False)
    print(models_df.to_string(index=False))

## Векторизация слов на основе ngrams и обучение модели классификации

### Векторизация слов на основе униграмм

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

unigram_vectorizer = CountVectorizer(ngram_range=(1, 1))
unigram_vectorized_x_train = unigram_vectorizer.fit_transform(x_train)
list(unigram_vectorizer.vocabulary_.items())[:5]

[('rt', 74601),
 ('craythornleonek', 22062),
 ('уже', 226778),
 ('рабочий', 199561),
 ('день', 124609)]

In [10]:
unigram_vectorized_x_train.shape

(170125, 244090)

In [11]:
unigram_clf = LogisticRegression(random_state=42, max_iter =1000)
unigram_clf.fit(unigram_vectorized_x_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [12]:
unigram_vectorized_x_test = unigram_vectorizer.transform(x_test)

In [13]:
unigram_pred = unigram_clf.predict(unigram_vectorized_x_test)
print(classification_report(y_test, unigram_pred))
save_model_result("Unigrams", accuracy_score(y_test, unigram_pred))

              precision    recall  f1-score   support

    negative       0.76      0.77      0.77     28101
    positive       0.77      0.77      0.77     28608

    accuracy                           0.77     56709
   macro avg       0.77      0.77      0.77     56709
weighted avg       0.77      0.77      0.77     56709



### Векторизация слов на основе триграмм

In [14]:
trigram_vectorizer = CountVectorizer(ngram_range=(3, 3))
trigram_vectorized_x_train = trigram_vectorizer.fit_transform(x_train)
list(trigram_vectorizer.vocabulary_.items())[:5]

[('rt craythornleonek уже', 133402),
 ('craythornleonek уже рабочий', 37191),
 ('уже рабочий день', 1203222),
 ('рабочий день почти', 981001),
 ('день почти подходит', 374738)]

In [15]:
trigram_vectorized_x_train.shape

(170125, 1329232)

In [16]:
trigram_clf = LogisticRegression(random_state=42, max_iter =1000)
trigram_clf.fit(trigram_vectorized_x_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [17]:
trigram_vectorized_x_test = trigram_vectorizer.transform(x_test)

In [18]:
trigram_pred = trigram_clf.predict(trigram_vectorized_x_test)
print(classification_report(y_test, trigram_pred))
save_model_result("Trigrams", accuracy_score(y_test, trigram_pred))

              precision    recall  f1-score   support

    negative       0.72      0.47      0.56     28101
    positive       0.61      0.82      0.70     28608

    accuracy                           0.64     56709
   macro avg       0.66      0.64      0.63     56709
weighted avg       0.66      0.64      0.63     56709



Большой разброс в точности между униграммами и триграммами может быть связан с тем, что триграммы резко увеличивают пространство уникальных признаков, что приводит к тому что очень сложно найти общие закономерности в данных.

## Векторизация слов на основе TF-IDF

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 5))
tfidf_vectorized_x_train = tfidf_vectorizer.fit_transform(x_train)

In [20]:
clf = LogisticRegression(random_state =42)
clf.fit(tfidf_vectorized_x_train , y_train)

LogisticRegression(random_state=42)

In [21]:
tfidf_vectorized_x_test = tfidf_vectorizer.transform(x_test)

pred = clf.predict(tfidf_vectorized_x_test)
print(classification_report(y_test , pred))

save_model_result("TF-IDF", accuracy_score(y_test, pred))

              precision    recall  f1-score   support

    negative       0.74      0.77      0.75     28101
    positive       0.76      0.73      0.75     28608

    accuracy                           0.75     56709
   macro avg       0.75      0.75      0.75     56709
weighted avg       0.75      0.75      0.75     56709



## Токенизация текста с помощью нескольких разных токенизаторов

In [22]:
import nltk
from nltk import tokenize

test_sentence = "This is a test sentence. Let's see how it works!"

In [23]:
tokenize.TreebankWordTokenizer().tokenize(test_sentence)

['This',
 'is',
 'a',
 'test',
 'sentence.',
 'Let',
 "'s",
 'see',
 'how',
 'it',
 'works',
 '!']

In [24]:
tokenize.simple.SpaceTokenizer().tokenize(test_sentence)

['This', 'is', 'a', 'test', 'sentence.', "Let's", 'see', 'how', 'it', 'works!']

In [25]:
tokenize.sonority_sequencing.SyllableTokenizer().tokenize(test_sentence.replace(' ', ''))

['Thi',
 'si',
 'sa',
 'tes',
 'tsen',
 'ten',
 'ce',
 '.',
 'Let',
 "'",
 'ssee',
 'ho',
 'wi',
 'tworks!']

## Стоп-слова и пунктуация

In [26]:
if FIRST_TIME:
    nltk.download('stopwords')
    nltk.download('punkt_tab')

from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize

to_remove = stopwords.words('russian') + list(punctuation)

no_punctuation_vectorizer = CountVectorizer(
    ngram_range=(1, 1),
    tokenizer=word_tokenize,
    stop_words=to_remove,
)

In [27]:
no_punctuation_vectorized_x_train = no_punctuation_vectorizer.fit_transform(x_train)

/home/glebmavi/.virtualenvs/Neiro/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/glebmavi/.virtualenvs/Neiro/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  warnings.warn(


In [28]:
no_punctuation_clf = LogisticRegression(random_state=42, max_iter=1000)
no_punctuation_clf.fit(no_punctuation_vectorized_x_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [29]:
no_punctuation_x_test = no_punctuation_vectorizer.transform(x_test)

In [30]:
no_punctuation_pred = no_punctuation_clf.predict(no_punctuation_x_test)
print(classification_report(y_test, no_punctuation_pred))
save_model_result("No punctuation", accuracy_score(y_test, no_punctuation_pred))

              precision    recall  f1-score   support

    negative       0.77      0.80      0.78     28101
    positive       0.79      0.76      0.78     28608

    accuracy                           0.78     56709
   macro avg       0.78      0.78      0.78     56709
weighted avg       0.78      0.78      0.78     56709



## Лемматизация

In [31]:
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

test_for_lemmatization = ['Если', 'б', 'мне', 'платили', 'каждый', 'раз']

for word in test_for_lemmatization:
    analyzed = pymorphy2_analyzer.parse(word)
    print(f"{analyzed} \n {analyzed[0].normal_form} \n")

[Parse(word='если', tag=OpencorporaTag('CONJ'), normal_form='если', score=1.0, methods_stack=((DictionaryAnalyzer(), 'если', 20, 0),))] 
 если 

[Parse(word='б', tag=OpencorporaTag('PRCL'), normal_form='б', score=0.763636, methods_stack=((DictionaryAnalyzer(), 'б', 22, 0),)), Parse(word='б', tag=OpencorporaTag('NOUN,inan,masc,Fixd sing,nomn'), normal_form='б', score=0.018181, methods_stack=((DictionaryAnalyzer(), 'б', 173, 0),)), Parse(word='б', tag=OpencorporaTag('NOUN,inan,masc,Fixd sing,gent'), normal_form='б', score=0.018181, methods_stack=((DictionaryAnalyzer(), 'б', 173, 1),)), Parse(word='б', tag=OpencorporaTag('NOUN,inan,masc,Fixd sing,datv'), normal_form='б', score=0.018181, methods_stack=((DictionaryAnalyzer(), 'б', 173, 2),)), Parse(word='б', tag=OpencorporaTag('NOUN,inan,masc,Fixd sing,accs'), normal_form='б', score=0.018181, methods_stack=((DictionaryAnalyzer(), 'б', 173, 3),)), Parse(word='б', tag=OpencorporaTag('NOUN,inan,masc,Fixd sing,ablt'), normal_form='б', score=0.0

## Эксплоративный анализ

In [32]:
alternative_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize)

alternative_tfidf_vectorized_x_train = alternative_tfidf_vectorizer.fit_transform(x_train)

/home/glebmavi/.virtualenvs/Neiro/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [33]:
clf = LogisticRegression(random_state=42)
clf.fit(alternative_tfidf_vectorized_x_train, y_train)

alternative_tfidf_vectorized_x_test = alternative_tfidf_vectorizer.transform(x_test)

pred = clf.predict(alternative_tfidf_vectorized_x_test)
print(classification_report(y_test, pred))

save_model_result("Alternative TF-IDF", accuracy_score(y_test, pred))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     28101
    positive       1.00      1.00      1.00     28608

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



## Регулярные выражения

In [34]:
import re

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [35]:
text = "Это пример строки, в которой есть несколько слов."

def get_first_two_letters(text):
    pattern = r'\b\w{1,2}'
    matches = re.findall(pattern, text)
    return matches

print(get_first_two_letters(text))

['Эт', 'пр', 'ст', 'в', 'ко', 'ес', 'не', 'сл']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.


In [36]:
text = "Это первое предложение. Это второе предложение. А это третье предложение. А это четвертое предложение."

def get_first_three_sentences(text):
    pattern = r'\.\s*'
    matches = re.split(pattern, text, maxsplit=2)
    return matches

get_first_three_sentences(text)

['Это первое предложение',
 'Это второе предложение',
 'А это третье предложение. А это четвертое предложение.']

**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [37]:
text = "В этом тексте есть 123 числа и 456789 цифры."

def replace_digits_with_dig(text):
    pattern = r'\d'
    replaced_text = re.sub(pattern, 'DIG', text)
    return replaced_text

replace_digits_with_dig(text)

'В этом тексте есть DIGDIGDIG числа и DIGDIGDIGDIGDIGDIG цифры.'

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [38]:
text = "Это пример строки, в которой есть несколько слов, которые длиннее трех символов xd."

def get_longer_than_three_chars(text):
    pattern = r'\b\w{3,}'
    matches = re.findall(pattern, text)
    return matches

get_longer_than_three_chars(text)

['Это',
 'пример',
 'строки',
 'которой',
 'есть',
 'несколько',
 'слов',
 'которые',
 'длиннее',
 'трех',
 'символов']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [39]:
text = "abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz"

def get_domains(text):
    pattern = r'@\w*\.\w*'
    matches = re.findall(pattern, text)
    return matches

get_domains(text)

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']

## Результаты моделей

In [40]:
show_ranked_models()

              name  accuracy
Alternative TF-IDF  0.998060
    No punctuation  0.781040
          Unigrams  0.767885
            TF-IDF  0.749669
          Trigrams  0.643813
